<a href="https://colab.research.google.com/github/ANUBHAV4646/PRODIGY_GAI_04/blob/main/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from PIL import Image
import torch
from torchvision import transforms, models
import torch.optim as optim
import os

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

def load_image(filename):
    image = Image.open(filename).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image.to(device)

def get_latest_file():
    files_list = [f for f in os.listdir() if os.path.isfile(f)]
    return max(files_list, key=os.path.getmtime)

# Upload and load content image
print("🔹 Upload CONTENT image:")
files.upload()
content_filename = get_latest_file()
print(f"Automatically selected CONTENT image: {content_filename}")
content = load_image(content_filename)

# Upload and load style image
print("\n🔹 Upload STYLE image:")
files.upload()
style_filename = get_latest_file()
print(f"Automatically selected STYLE image: {style_filename}")
style = load_image(style_filename)

print("\n✅ Images loaded successfully!")
print(f"Content image tensor shape: {content.shape}")
print(f"Style image tensor shape: {style.shape}")

# Load VGG19 model
vgg = models.vgg19(pretrained=True).features.to(device).eval()

# Feature extraction and Gram matrix
def get_features(image, model):
    layers = {
        '0': 'conv1_1',
        '5': 'conv2_1',
        '10': 'conv3_1',
        '19': 'conv4_1',
        '21': 'content'
    }
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(c, h * w)
    return torch.mm(features, features.t())

# Feature extraction and initialization
content_features = get_features(content, vgg)
style_features = get_features(style, vgg)
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

target = content.clone().requires_grad_(True).to(device)
optimizer = optim.Adam([target], lr=0.01)

print("Setup complete. Ready for style transfer optimization!")


In [ ]:
# Compute content and style features without tracking gradients
with torch.no_grad():
    content_features = get_features(content, vgg)
    style_features = get_features(style, vgg)
    style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# Initialize target and optimizer
target = content.clone().requires_grad_(True).to(device)
optimizer = torch.optim.Adam([target], lr=0.01)

# Hyperparameters
num_steps = 300
content_weight = 1e4
style_weight = 1e2

# Optimization loop
for step in range(1, num_steps + 1):
    optimizer.zero_grad()

    target_features = get_features(target, vgg)

    content_loss = torch.mean((target_features['content'] - content_features['content']) ** 2)

    style_loss = 0
    for layer in style_grams:
        target_gram = gram_matrix(target_features[layer])
        style_gram = style_grams[layer]
        style_loss += torch.mean((target_gram - style_gram) ** 2)

    total_loss = content_weight * content_loss + style_weight * style_loss
    total_loss.backward()
    optimizer.step()

    if step % 50 == 0:
        print(f"Step [{step}/{num_steps}], Content Loss: {content_loss.item():.4f}, Style Loss: {style_loss.item():.4f}, Total Loss: {total_loss.item():.4f}")


In [ ]:
import torchvision.transforms.functional as TF
from torchvision.utils import save_image
from datetime import datetime
from IPython.display import display
from google.colab import files
import os

# Style transfer optimization
for step in range(1, num_steps + 1):
    optimizer.zero_grad()
    target_features = get_features(target, vgg)

    content_loss = torch.mean((target_features['content'] - content_features['content']) ** 2)

    style_loss = 0
    for layer in style_grams:
        target_gram = gram_matrix(target_features[layer])
        style_gram = style_grams[layer]
        style_loss += torch.mean((target_gram - style_gram) ** 2)

    total_loss = content_weight * content_loss + style_weight * style_loss
    total_loss.backward()
    optimizer.step()

    if step % 50 == 0:
        print(f"Step [{step}/{num_steps}], Content Loss: {content_loss.item():.4f}, Style Loss: {style_loss.item():.4f}, Total Loss: {total_loss.item():.4f}")

# Convert tensor to PIL image
def im_convert(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze(0)
    return TF.to_pil_image(image)

# Display final image
final_image = im_convert(target)
display(final_image)

# Save and download image
filename = f"stylized_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
save_image(target.clamp(0, 1), filename)
print(f"✅ Stylized image saved as {filename}")
files.download(filename)
